In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name']
)

mycursor = mydb.cursor()

In [4]:
mycursor = mydb.cursor()

 
query = "SELECT id FROM profile_info WHERE last_active_date > '2023-12-12' ORDER BY last_active_date DESC limit 300"

 

mycursor.execute(query)

result = mycursor.fetchall()


In [5]:
udf = pd.DataFrame(result,columns=['userid'])

In [6]:
uid_list = list(udf.userid)

In [7]:
# uid_list

In [8]:
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string
     

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)

    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        # try:
        response = requests.request("POST", url, headers=headers, data = str(payload))
        response_json = response.json()

        res.append(response_json['docs']) 

        # except Exception as e:
        #     continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'resume' : []
    }

    for s in tqdm(user_json):

        try:
            source = s[0]

            user_dict['id'].append(source['id'])
            user_dict['resume'].append(clean_space(source['resumeText']))

        except Exception as e:
            continue

    return user_dict

In [9]:

user_df = get_user_data_search_embed(uid_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 4597.54it/s]


In [10]:
u1 = user_df.sample(5)

In [11]:
u1

,id,resume
70,1674126,"FINANCE EXECUTIVE AT NOON, UAE mansi91midha@gm..."
93,2790216,Educational Qualification Cource University Pe...
83,2149380,/zahabiyakinkhabwala zahabiyak15@gmail.com CON...
223,2579933,Educational Qualification +91 8933949349 canis...
272,2784240,PADMAJA PENTYALA Mail-ID:p.padmaja129@gmail.co...


In [14]:
# /home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data/

from fpdf import FPDF

output_directory = '/home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data_pdf/'

# for index, row in tqdm(user_df.iterrows()):
#     user_id = row['id']
#     resume_text = row['resume']

#     # Creating the filename
#     filename = f"{output_directory}uid_{user_id}.txt"

#     # Writing user id and resume text to the file
#     with open(filename, 'w', encoding='utf-8') as file:
#         file.write(f"User ID: {user_id}\n")
#         file.write(resume_text)

# print("Files saved successfully.")




for index, row in tqdm(user_df.iterrows()):
    user_id = row['id']
    resume_text = row['resume']

    # Creating the filename for PDF
    pdf_filename = f"{output_directory}uid_{user_id}.pdf"

    # Creating a PDF instance
    pdf = FPDF()
    pdf.add_page()

    # Setting font
    pdf.set_font("Arial", size=12)

    # Adding user id and resume text to the PDF
    pdf.cell(200, 10, txt=f"User ID: {user_id}", ln=True, align='L')
    pdf.multi_cell(200, 10, txt=resume_text)

    # Save the PDF to the file
    pdf.output(pdf_filename)

print("PDF files saved successfully.")

277it [00:00, 395.58it/s]

PDF files saved successfully.


In [19]:
user_df.loc[user_df['id']=='1874555']

,id,resume
139,1874555,Surbhi Anand Worked as an Assistant Sales Mana...


## Direct RAG example

In [3]:
openai.api_key=config['key']['infoedge']



In [4]:
os.environ['OPENAI_API_KEY'] = openai.api_key

In [5]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import TextLoader


loader = TextLoader("/home/ubuntu/infoedge/llama-recipes/examples/custom_data/user_resume_data/uid_1103243.txt")
index = VectorstoreIndexCreator().from_loaders([loader])
index.query("What are relationship manager career objectives")

' The career objective of a relationship manager is to pursue an ambitious career in an organization where there are ample learning opportunities, upward mobility, exposure to innovation, and motivation. They aim to exhibit their resourcefulness in both technical and organizational skills and to take on challenging roles in key account management, travel and expense management solutions, and relationship management.'